In [1]:
import sys
sys.path.insert(0, "/home/dzigen/Desktop/ITMO/ВКР/КМУ2024/inference.ipynb")

from src.retrievers.bm25colbert import BM25ColBertRetriever
from src.retrievers.bm25e5 import BM25E5Retriever
from src.readers.fid import FiDReader
from src.retrievers.e5 import E5Retriever

import torch
from tqdm import tqdm

In [2]:
TUNED_READER_PATH = '/home/dzigen/Desktop/ITMO/ВКР/КМУ2024/logs/join_e5_fid_triviaqa/reader_bestmodel.pt'
BASE_PATH = '/home/dzigen/Desktop/ITMO/ВКР/КМУ2024/data/bases/e5_scipdf_base'
READER_INPUT_FORMAT = "context: {c}\n\nquestion: {q}"
READER_GEN_ML = 256

QUESTIONS = [
    "What is a RETRO approach",
    "What is a kNN-LM approach",
    "What is a DPR approach",
    "What is a RAG approach",
    "What is a FiD approach",
    "What is a EMDR2 approach",
    "What is a Atlas approach",
    "What is a REPLUG approach",
    "What is a ColBERT approach"
]

In [3]:
def inference(reader, retriever, queries):
    answers = []
    for query in tqdm(queries):
        print("QUERY: ", query)
        texts, k_scores, metadata = retriever.search(query)

        print("CONTEXTS:\n", '\n\n'.join(texts))

        formated_txts = list(map(
            lambda t: READER_INPUT_FORMAT.format(q=query,c=t), texts))

        tokenized_txts = reader.tokenize(formated_txts)
        
        cands_k = len(texts)

        # Generating Answers by predicted indices
        output = reader.model.generate(
            input_ids=tokenized_txts['input_ids'].view(1, cands_k, -1),
            attention_mask=tokenized_txts['attention_mask'].view(1, cands_k, -1), 
            max_length=READER_GEN_ML, eos_token_id=reader.tokenizer.eos_token_id)
        
        predicted = reader.tokenizer.batch_decode(output, skip_special_tokens=True)
        answers += predicted

        print("ANSWER: ", predicted[0])

    return answers

In [5]:
reader = FiDReader()
reader.model.load_state_dict(torch.load(TUNED_READER_PATH))
#reader.load_model(TUNED_READER_PATH)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


#### E5 + FID

In [9]:
TUNED_RETRIEVER_PATH = '/home/dzigen/Desktop/ITMO/ВКР/КМУ2024/logs/join_e5_fid_triviaqa/retriever_bestmodel.pt'

e5_retriever = E5Retriever()
e5_retriever.model.load_state_dict(torch.load(TUNED_RETRIEVER_PATH))
#colb_retriever.load_model(TUNED_RETRIEVER_PATH)
e5_retriever.load_base(BASE_PATH)

Loading query E5-model...
Loading document E5-model...
Loading precomputed e5-base...


`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


In [12]:
inference(reader, e5_retriever, QUESTIONS)

  0%|          | 0/9 [00:00<?, ?it/s]

QUERY:  What is a RETRO approach
CONTEXTS:
 Shall We Pretrain Autoregressive Language Models with Retrieval? A Comprehensive Study
To answer the above question and bridge the missing gap, we perform an extensive study on RETRO, as to the best of our knowledge, RETRO is the only retrieval-augmented autoregressive LM that supports large-scale pretraining with retrieval on the massive pretraining corpus with hundreds of billion or trillion tokens.

Shall We Pretrain Autoregressive Language Models with Retrieval? A Comprehensive Study
These results further substantiate the potential of RETRO, which is pre-trained with retrieval capabilities, as a promising approach.

Improving language models by retrieving
from trillions of tokens
Retro models are ﬂexible and can be used without retrieval at evaluation and still achieve comparable performance to baseline models.

Shall We Pretrain Autoregressive Language Models with Retrieval? A Comprehensive Study
For open-domain QA tasks, RETRO achieves 

 11%|█         | 1/9 [00:02<00:21,  2.73s/it]

ANSWER:  Retrieval
QUERY:  What is a kNN-LM approach
CONTEXTS:
 GENERALIZATION THROUGH MEMORIZATION : NEAREST NEIGHBOR LANGUAGE MODELS
We introduce kNN-LM, an approach that extends a pre-trained LM by linearly interpolating its next word distribution with a k-nearest neighbors (kNN) model.

COPY IS ALL YOU NEED
• kNN-LM (Khandelwal et al., 2020) is a retrieval-augmented generation model, which extends a pre-trained neural language model by linearly interpolating its next token distribution with a k-nearest neighbors (kNN) model.

GENERALIZATION THROUGH MEMORIZATION : NEAREST NEIGHBOR LANGUAGE MODELS
We introduce kNN-LMs, which extend a pre-trained neural language model (LM) by linearly interpolating it with a k-nearest neighbors (kNN) model.

GENERALIZATION THROUGH MEMORIZATION : NEAREST NEIGHBOR LANGUAGE MODELS
The kNN-LM involves augmenting such a pre-trained LM with a nearest neighbors retrieval mechanism, without any additional training (the representations learned by the LM remain

 22%|██▏       | 2/9 [00:03<00:10,  1.56s/it]

...

 89%|████████▉ | 8/9 [00:09<00:00,  1.00it/s]

ANSWER:  Retrieval-Augmented Black-Box Language Models
QUERY:  What is a ColBERT approach
CONTEXTS:
 ColBERT: Efficient and Effective Passage Search via Contextualized Late Interaction over BERT
ColBERT prescribes a simple framework for balancing the quality and cost of neural IR, particularly deep language models like BERT.

ColBERT: Efficient and Effective Passage Search via Contextualized Late Interaction over BERT
Recall that ColBERT can be used for re-ranking the output of another retrieval model, typically a term-based model, or directly for end-to-end retrieval from a document collection.

ColBERT: Efficient and Effective Passage Search via Contextualized Late Interaction over BERT
To reconcile efciency and contextualization in IR, we propose ColBERT, a ranking model based on contextualized late interaction over BERT.

ColBERT: Efficient and Effective Passage Search via Contextualized Late Interaction over BERT
In this paper, we introduced ColBERT, a novel ranking model that emp

100%|██████████| 9/9 [00:09<00:00,  1.09s/it]

ANSWER:  Efficient and Effective Passage Search


['Retrieval',
 'We introduce kNN-LMs',
 'Dense Passage Retrieval',
 'ad hoc approach',
 'Achieves State of the Art Results Compared to Similar sized Models',
 'A Framework for End-to-End Training of Multi-Document Reader and Retriever for Open-Domain Question Answering',
 'Atlas: Few-shot Learning with Retrieval Augmented Language Models',
 'Retrieval-Augmented Black-Box Language Models',
 'Efficient and Effective Passage Search']

#### BM25ColBERT + FID

In [ ]:
TUNED_RETRIEVER_PATH = ''

colb_retriever = BM25ColBertRetriever()
colb_retriever.load_model(TUNED_RETRIEVER_PATH)
colb_retriever.load_base(BASE_PATH)

In [ ]:
inference(reader, colb_retriever, QUESTIONS)

#### BM25E5 + FID

In [ ]:
TUNED_RETRIEVER_PATH = ''

e5_retriever = BM25ColBertRetriever()
e5_retriever.load_model(TUNED_RETRIEVER_PATH)
e5_retriever.load_base(BASE_PATH)

In [ ]:
inference(reader, e5_retriever, QUESTIONS)